In [43]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

from sklearn import metrics
from sklearn.model_selection import train_test_split

## Data

In [44]:
# Remove aggregate rows, replace NaN with 0

puf = pd.read_csv('puf2011.csv')

puf = puf[(puf['RECID'] != 999996) &
          (puf['RECID'] != 999997) &
          (puf['RECID'] != 999998) &
          (puf['RECID'] != 999999)
         ]
           
puf = puf.fillna(0)

#  MARS to k-1 dummies

puf[['MARS2', 'MARS3', 'MARS4']] = pd.get_dummies(puf['MARS'], drop_first = True)

# These vars are combined in cps

puf['E19800_E20100'] = puf['E19800'] + puf['E20100']

# All vars shared between puf and cps except E00650 (colinear w/E00600) and E01100 (crashing mnlogit)

potential_pred =  [
                  'DSI', 'EIC', 'MARS2', 'MARS3', 'MARS4', 'XTOT', 'E00200', 
                  'E00300', 'E00400', 'E00600', 'E00800', 'E00900', 'E01400', 
                  'E01500', 'E01700', 'E02100', 'E02300', 'E02400', 'E03150', 
                  'E03210', 'E03240', 'E03270', 'E03300', 'E17500', 'E18400', 
                  'E18500', 'E19200', 'E19800_E20100','E20400', 'E32800', 
                  'F2441', 'N24'
                   ]
        
# Log of response       
        
puf['log_P22250'] = np.where(puf['P22250'] == 0, 0, np.sign(puf['P22250'])*np.log(abs(puf['P22250'])))

keep = ['P22250', 'AGIR1', 'log_P22250'] + potential_pred

puf = puf[keep]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in log


## Train / test

In [45]:
np.random.seed(100)
train, test = train_test_split(puf, test_size=0.2)

# Subsamples of train/test where P22250 > 0, or < 0 pos or neg for 2nd stage imputation

pos_train = train.copy()[train.copy()['P22250'] > 0]
neg_train = train.copy()[train.copy()['P22250'] < 0]

pos_test =  test.copy()[test.copy()['P22250'] > 0]
neg_test =  test.copy()[test.copy()['P22250'] < 0]

## Models

OLS for positive data

Using intuitively relevant predictors with low p-values, and low inter-predictor-collinearity

In [46]:
pred_pos =  [
            'DSI', 'EIC', 'MARS2', 'MARS3', 'MARS4', 'XTOT', 'E00200',
            'E00300', 'E00400', 'E00600', 'E03240',
            'E18400', 'E18500'
            ]

endog_pos = pos_train['log_P22250']
exog_pos = pos_train[pred_pos]

fit_pos = sm.OLS(endog_pos, exog_pos).fit()
summary_pos = fit_pos.summary()

# Predictions of log(P22250), transforming back to P22250

log_predictions_pos = fit_pos.predict(pos_test[pred_pos])
predictions_pos = np.exp(log_predictions_pos)
max_predictions_pos = round(predictions_pos.max(), 2)

actual_pos = pos_test['P22250']

MSE_pos = round(metrics.mean_squared_error(actual_pos, predictions_pos),2)

print(
     'Largest predicted P22250: $' + '{:,}'.format(max_predictions_pos), '\n',
     'MSE is: ' + str(MSE_pos), '\n',
      summary_pos
     ) 

Largest predicted P22250: $559,038,014,760.08 
 MSE is: 9.003850794399043e+19 
                             OLS Regression Results                            
Dep. Variable:             log_P22250   R-squared:                       0.791
Model:                            OLS   Adj. R-squared:                  0.790
Method:                 Least Squares   F-statistic:                     4586.
Date:                Thu, 26 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:10:07   Log-Likelihood:                -44044.
No. Observations:               15800   AIC:                         8.811e+04
Df Residuals:                   15787   BIC:                         8.821e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------

Actual largest P22250 values are in e+7 range

In [47]:
puf['P22250'].sort_values(ascending=False)[:11]

152224    39410000.0
132670    39400000.0
67871     32940000.0
160082    25410000.0
158722    21750000.0
151429    20980000.0
160072    18170000.0
161329    15340000.0
143041    13560000.0
155360    13520000.0
144883    13040000.0
Name: P22250, dtype: float64

Largest predicted values are much larger

In [48]:
predictions_pos.sort_values(ascending=False)[:11]

162832    5.590380e+11
159922    1.846003e+11
158769    7.150886e+10
138866    6.751121e+09
163428    4.607185e+09
150231    1.156800e+09
149696    4.950723e+08
162028    1.929891e+08
158876    7.016240e+07
162164    5.981978e+07
162831    5.865267e+07
dtype: float64

Summary stats

In [49]:
print(
      'Mean of actual positive P22250 testing data: ' + str(pos_test['P22250'].mean()), '\n',
      'Mean of predicted positive P22250 testing data: ' + str(predictions_pos.mean()), '\n',
      'Actual median, std dev: ' + str(pos_test['P22250'].median()) + ', ' + str(pos_test['P22250'].std()), '\n',
      'Predicted median, std dev: '  + str(predictions_pos.median()) + ', ' + str(predictions_pos.std())
     )

Mean of actual positive P22250 testing data: 96625.79933452776 
 Mean of predicted positive P22250 testing data: 212100579.44832927 
 Actual median, std dev: 4890.0, 887552.8410844681 
 Predicted median, std dev: 1471.0836236939317, 9487736751.303223


Similar issue (but much worse) with different mix of continuous/discrete predictors

In [50]:
pred_pos =  [
            'MARS2', 'MARS3', 'MARS4', 'N24', 'E00200',
            'E00600', 'E02100', 'E18400', 'E18500', 'E19200', 'E19800_E20100', 'E32800'
            ]

# Changed to float as sm.OLS was throwing exception

endog_pos = pos_train['log_P22250']
exog_pos = pos_train[pred_pos]

fit_pos = sm.OLS(endog_pos, exog_pos).fit()
summary_pos = fit_pos.summary()

# Changed back to int as .predict() was throwing exception

log_predictions_pos = fit_pos.predict(pos_test[pred_pos])
predictions_pos = np.exp(log_predictions_pos)
max_predictions_pos = round(predictions_pos.max(), 2)

actual_pos = pos_test['P22250']

MSE_pos = round(metrics.mean_squared_error(actual_pos, predictions_pos),2)

print(
     'Largest predicted P22250: $' + '{:,}'.format(max_predictions_pos), '\n',
     'MSE is: ' + str(MSE_pos), '\n',
      summary_pos
     )

Largest predicted P22250: $1.0531054189615383e+19 
 MSE is: 2.838574413733693e+34 
                             OLS Regression Results                            
Dep. Variable:             log_P22250   R-squared:                       0.770
Model:                            OLS   Adj. R-squared:                  0.770
Method:                 Least Squares   F-statistic:                     4413.
Date:                Thu, 26 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:10:07   Log-Likelihood:                -44775.
No. Observations:               15800   AIC:                         8.957e+04
Df Residuals:                   15788   BIC:                         8.967e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------

Only including discrete predictors greatly underestimates P22250

In [51]:
pred_pos =  [
            'EIC', 'DSI', 'MARS2', 'MARS3', 'MARS4',
            'XTOT'
            ]

# Changed to float as sm.OLS was throwing exception

endog_pos = pos_train['log_P22250']
exog_pos = pos_train[pred_pos]

fit_pos = sm.OLS(endog_pos, exog_pos).fit()
summary_pos = fit_pos.summary()

# Changed back to int as .predict() was throwing exception

log_predictions_pos = fit_pos.predict(pos_test[pred_pos])
predictions_pos = np.exp(log_predictions_pos)
max_predictions_pos = round(predictions_pos.max(), 2)

actual_pos = pos_test['P22250']

MSE_pos = round(metrics.mean_squared_error(actual_pos, predictions_pos),2)

print(
     'Largest predicted P22250: $' + '{:,}'.format(max_predictions_pos), '\n',
     'MSE is: ' + str(MSE_pos), '\n',
      summary_pos
     )

Largest predicted P22250: $31,864.26 
 MSE is: 795348986841.39 
                             OLS Regression Results                            
Dep. Variable:             log_P22250   R-squared:                       0.768
Model:                            OLS   Adj. R-squared:                  0.768
Method:                 Least Squares   F-statistic:                     8699.
Date:                Thu, 26 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:10:07   Log-Likelihood:                -44866.
No. Observations:               15800   AIC:                         8.974e+04
Df Residuals:                   15794   BIC:                         8.979e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

Summary stats

In [52]:
print(
      'Mean of actual positive P22250 testing data: ' + str(pos_test['P22250'].mean()), '\n',
      'Mean of predicted positive P22250 testing data: ' + str(predictions_pos.mean()), '\n',
      'Actual median, std dev: ' + str(pos_test['P22250'].median()) + ', ' + str(pos_test['P22250'].std()), '\n',
      'Predicted median, std dev: '  + str(predictions_pos.median()) + ', ' + str(predictions_pos.std())
     )

Mean of actual positive P22250 testing data: 96625.79933452776 
 Mean of predicted positive P22250 testing data: 6411.195653107481 
 Actual median, std dev: 4890.0, 887552.8410844681 
 Predicted median, std dev: 945.3514463125927, 9934.646111825858


Including just 1 continuous predictor again overestimates

In [53]:
pred_pos =  [
            'EIC', 'DSI', 'MARS2', 'MARS3', 'MARS4',
            'XTOT', 'E00300'
            ]

# Changed to float as sm.OLS was throwing exception

endog_pos = pos_train['log_P22250']
exog_pos = pos_train[pred_pos]

fit_pos = sm.OLS(endog_pos, exog_pos).fit()
summary_pos = fit_pos.summary()

# Changed back to int as .predict() was throwing exception

log_predictions_pos = fit_pos.predict(pos_test[pred_pos])
predictions_pos = np.exp(log_predictions_pos)
max_predictions_pos = round(predictions_pos.max(), 2)

actual_pos = pos_test['P22250']

MSE_pos = round(metrics.mean_squared_error(actual_pos, predictions_pos),2)

print(
     'Largest predicted P22250: $' + '{:,}'.format(max_predictions_pos), '\n',
     'MSE is: ' + str(MSE_pos), '\n',
      summary_pos
     )

Largest predicted P22250: $1,186,290,075.72 
 MSE is: 433177998947119.8 
                             OLS Regression Results                            
Dep. Variable:             log_P22250   R-squared:                       0.774
Model:                            OLS   Adj. R-squared:                  0.774
Method:                 Least Squares   F-statistic:                     7737.
Date:                Thu, 26 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:10:07   Log-Likelihood:                -44640.
No. Observations:               15800   AIC:                         8.929e+04
Df Residuals:                   15793   BIC:                         8.935e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------

Summary stats

In [54]:
print(
      'Mean of actual positive P22250 testing data: ' + str(pos_test['P22250'].mean()), '\n',
      'Mean of predicted positive P22250 testing data: ' + str(predictions_pos.mean()), '\n',
      'Actual median, std dev: ' + str(pos_test['P22250'].median()) + ', ' + str(pos_test['P22250'].std()), '\n',
      'Predicted median, std dev: '  + str(predictions_pos.median()) + ', ' + str(predictions_pos.std())
     )

Mean of actual positive P22250 testing data: 96625.79933452776 
 Mean of predicted positive P22250 testing data: 467270.05469735333 
 Actual median, std dev: 4890.0, 887552.8410844681 
 Predicted median, std dev: 986.7399972255719, 21120206.53086928
